In [3]:
import torch
import os 
import sys
import umap, numba
import numpy as np
import tqdm
import glob, pandas as pd
import pickle
import fire
import scipy.sparse as sps
from visdom import Visdom
from sklearn.metrics import f1_score
import copy
import torch.nn.functional as F
from torch import nn
from sklearn.preprocessing import LabelEncoder
import torch_cluster
from torch_cluster import knn_graph
from torch_geometric.data import Data 
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import InMemoryDataset,DataLoader
from torch_geometric.utils import add_remaining_self_loops
from torch_cluster import radius_graph
from torch_geometric.utils import subgraph

2022-12-20 22:34:24.333106: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
IDs=['109_A1c_ASAP_tumor_map', '10_A1a_ASAP_tumor_map', '10_A1b_ASAP_tumor_map', '10_A2b_ASAP_tumor_map', '110_A2b_ASAP_tumor_map', '112_a_ASAP_tumor_map', '112_b_ASAP_tumor_map', '123_A1a_ASAP_tumor_map', '12_A1c_ASAP_tumor_map', '14_A1b_ASAP_tumor_map', '14_A2b_ASAP_tumor_map', '169_A2b_ASAP_tumor_map', '270_A1b_ASAP_tumor_map', '270_A1d_ASAP_tumor_map', '270_A1e_ASAP_tumor_map', '270_A2b_ASAP_tumor_map', '270_A2f_ASAP_tumor_map', '281_A1d_ASAP_tumor_map', '281_A1f_ASAP_tumor_map', '281_A2eX_ASAP_tumor_map', '311_A2c_ASAP_tumor_map', '327_A1a_ASAP_tumor_map', '327_A1d_ASAP_tumor_map', '327_B1c_ASAP_tumor_map', '341_a_ASAP_tumor_map', '341_b_ASAP_tumor_map', '342_a_ASAP_tumor_map', '342_b_ASAP_tumor_map', '343_a_ASAP_tumor_map', '343_b_ASAP_tumor_map', '343_c_ASAP_tumor_map', '343_d_ASAP_tumor_map', '344_a_ASAP_tumor_map', '344_b_ASAP_tumor_map', '345_a_ASAP_tumor_map', '345_b_ASAP_tumor_map', '346_a_ASAP_tumor_map', '346_b_ASAP_tumor_map', '350_A1a_ASAP_tumor_map', '350_A1b_ASAP_tumor_map', '350_A1c_ASAP_tumor_map', '350_A1d_ASAP_tumor_map', '350_A1e_ASAP_tumor_map', '351_A2b_ASAP_tumor_map', '352_A1d_ASAP_tumor_map', '352_A1e_ASAP_tumor_map', '352_A1g_ASAP_tumor_map', '352_A1h_ASAP_tumor_map', '352_A1i_ASAP_tumor_map', '353_A2b_ASAP_tumor_map', '354_A1b_ASAP_tumor_map', '354_A1c_ASAP_tumor_map', '354_A1d_ASAP_tumor_map', '354_A3a_ASAP_tumor_map', '354_A3b_ASAP_tumor_map', '354_A3c_ASAP_tumor_map', '354_D1b_ASAP_tumor_map', '355_A1d_ASAP_tumor_map', '356_A1b_ASAP_tumor_map', '358_A1a_ASAP_tumor_map', '358_A1b_ASAP_tumor_map', '361_a_ASAP_tumor_map', '361_b_ASAP_tumor_map', '362_A1a_ASAP_tumor_map', '362_A1b_ASAP_tumor_map', '362_A1c_ASAP_tumor_map', '363_A1b_ASAP_tumor_map', '363_A1c_ASAP_tumor_map', '363_A2b_ASAP_tumor_map', '363_A3b_ASAP_tumor_map', '364_A1b_ASAP_tumor_map', '364_A2b_ASAP_tumor_map', '364_A4b_ASAP_tumor_map', '365_A1b_ASAP_tumor_map', '365_A2b_ASAP_tumor_map', '366_A1a_ASAP_tumor_map', '366_A1b_ASAP_tumor_map', '366_A1c_ASAP_tumor_map', '367_A2b_ASAP_tumor_map', '368_A1b_ASAP_tumor_map', '368_A1c_ASAP_tumor_map', '368_A1d_ASAP_tumor_map', '369_A1b_ASAP_tumor_map', '369_A1c_ASAP_tumor_map', '369_A2b_ASAP_tumor_map', '370_A1b_ASAP_tumor_map', '370_A2a_ASAP_tumor_map', '370_A2b_ASAP_tumor_map', '37_A2d_ASAP_tumor_map', '61_A1a_ASAP_tumor_map', '61_B1a_ASAP_tumor_map', '70_A2b_ASAP_tumor_map', '7_A1c_ASAP_tumor_map', '7_A1d_ASAP_tumor_map', '7_A1e_ASAP_tumor_map']

In [5]:

#TUMOR_PERCENT=0.2

def get_graph_dataset(embedding_dir='', 
                      basename='', 
                      target_col='scc',  
                      k=8,
                      radius=0,
                      dataset_save_loc=''):
    with open(os.path.join(embedding_dir,f"{basename}"),"rb") as file:
        embeddings=torch.load(file) 
    df=embeddings['patch_info']
    X=pd.DataFrame(embeddings['embeddings'])
    df['y_true']=df[target_col]
    xy=torch.tensor(embeddings['patch_info'][['x','y']].values)
    X=torch.tensor(X.values)
    y=torch.tensor(df.loc[:,'y_true'].values)
    
    if not radius:
        G=knn_graph(xy,k=k)
    else:
        G=radius_graph(xy, r=radius*np.sqrt(2), batch=None, loop=True)
    G=(add_remaining_self_loops(G)[0]).detach().cpu()
    xy=xy.detach().cpu()
    datasets=[]
    edges=G.detach().cpu().numpy().astype(int)
    n_components,components=list(sps.csgraph.connected_components(sps.coo_matrix((np.ones_like(edges[0]),(edges[0],edges[1])))))
    components=torch.LongTensor(components)
    for i in range(n_components):
        G_new=subgraph(components==i,G,relabel_nodes=True)[0]
        xy_new=xy[components==i]
        X_new=X[components==i]
        y_new=y[components==i]
        #y_whole_slide_image_label=y_new.float().mean()>=TUMOR_PERCENT
        np.random.seed(42)
        idx=np.arange(X_new.shape[0])
        idx2=np.arange(X_new.shape[0])
        np.random.shuffle(idx)
        train_idx,val_idx,test_idx=torch.tensor(np.isin(idx2,idx[:int(0.8*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.8*len(idx)):int(0.9*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.9*len(idx)):]))
        dataset=Data(x=X_new, edge_index=G_new, edge_attr=None, y=y_new, pos=xy_new)
        dataset.train_mask=train_idx
        dataset.val_mask=val_idx
        dataset.test_mask=test_idx
        datasets.append(dataset)
    #components=components.numpy()
    

    return datasets
    torch.save(datasets,dataset_save_loc)



In [45]:

get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/",
                  basename='109_A1c_ASAP_tumor_map' +'.pkl',
                  target_col='scc',
                  k=8,
                  radius=256,
                  dataset_save_loc=f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/112_a_ASAP_tumor_map.pkl")

[Data(x=[20589, 2048], edge_index=[2, 102219], y=[20589], pos=[20589, 2], train_mask=[20589], val_mask=[20589], test_mask=[20589]),
 Data(x=[20706, 2048], edge_index=[2, 102704], y=[20706], pos=[20706, 2], train_mask=[20706], val_mask=[20706], test_mask=[20706])]

In [37]:
IDs=['109_A1c_ASAP_tumor_map', '10_A1a_ASAP_tumor_map', '10_A1b_ASAP_tumor_map', '10_A2b_ASAP_tumor_map', '110_A2b_ASAP_tumor_map', '112_a_ASAP_tumor_map', '112_b_ASAP_tumor_map', '123_A1a_ASAP_tumor_map', '12_A1c_ASAP_tumor_map', '14_A1b_ASAP_tumor_map', '14_A2b_ASAP_tumor_map', '169_A2b_ASAP_tumor_map', '270_A1b_ASAP_tumor_map', '270_A1d_ASAP_tumor_map', '270_A1e_ASAP_tumor_map', '270_A2b_ASAP_tumor_map', '270_A2f_ASAP_tumor_map', '281_A1d_ASAP_tumor_map', '281_A1f_ASAP_tumor_map', '281_A2eX_ASAP_tumor_map', '311_A2c_ASAP_tumor_map', '327_A1a_ASAP_tumor_map', '327_A1d_ASAP_tumor_map', '327_B1c_ASAP_tumor_map', '341_a_ASAP_tumor_map', '341_b_ASAP_tumor_map', '342_a_ASAP_tumor_map', '342_b_ASAP_tumor_map', '343_a_ASAP_tumor_map', '343_b_ASAP_tumor_map', '343_c_ASAP_tumor_map', '343_d_ASAP_tumor_map', '344_a_ASAP_tumor_map', '344_b_ASAP_tumor_map', '345_a_ASAP_tumor_map', '345_b_ASAP_tumor_map', '346_a_ASAP_tumor_map', '346_b_ASAP_tumor_map', '350_A1a_ASAP_tumor_map', '350_A1b_ASAP_tumor_map', '350_A1c_ASAP_tumor_map', '350_A1d_ASAP_tumor_map', '350_A1e_ASAP_tumor_map', '351_A2b_ASAP_tumor_map', '352_A1d_ASAP_tumor_map', '352_A1e_ASAP_tumor_map', '352_A1g_ASAP_tumor_map', '352_A1h_ASAP_tumor_map', '352_A1i_ASAP_tumor_map', '353_A2b_ASAP_tumor_map', '354_A1b_ASAP_tumor_map', '354_A1c_ASAP_tumor_map', '354_A1d_ASAP_tumor_map', '354_A3a_ASAP_tumor_map', '354_A3b_ASAP_tumor_map', '354_A3c_ASAP_tumor_map', '354_D1b_ASAP_tumor_map', '355_A1d_ASAP_tumor_map', '356_A1b_ASAP_tumor_map', '358_A1a_ASAP_tumor_map', '358_A1b_ASAP_tumor_map', '361_a_ASAP_tumor_map', '361_b_ASAP_tumor_map', '362_A1a_ASAP_tumor_map', '362_A1b_ASAP_tumor_map', '362_A1c_ASAP_tumor_map', '363_A1b_ASAP_tumor_map', '363_A1c_ASAP_tumor_map', '363_A2b_ASAP_tumor_map', '363_A3b_ASAP_tumor_map', '364_A1b_ASAP_tumor_map', '364_A2b_ASAP_tumor_map', '364_A4b_ASAP_tumor_map', '365_A1b_ASAP_tumor_map', '365_A2b_ASAP_tumor_map', '366_A1a_ASAP_tumor_map', '366_A1b_ASAP_tumor_map', '366_A1c_ASAP_tumor_map', '367_A2b_ASAP_tumor_map', '368_A1b_ASAP_tumor_map', '368_A1c_ASAP_tumor_map', '368_A1d_ASAP_tumor_map', '369_A1b_ASAP_tumor_map', '369_A1c_ASAP_tumor_map', '369_A2b_ASAP_tumor_map', '370_A1b_ASAP_tumor_map', '370_A2a_ASAP_tumor_map', '370_A2b_ASAP_tumor_map', '37_A2d_ASAP_tumor_map', '61_A1a_ASAP_tumor_map', '61_B1a_ASAP_tumor_map', '70_A2b_ASAP_tumor_map', '7_A1c_ASAP_tumor_map', '7_A1d_ASAP_tumor_map', '7_A1e_ASAP_tumor_map']

for ID in tqdm.tqdm(IDs):
    get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/",
                  basename=ID +'.pkl',
                  target_col='scc',
                  k=8,
                  radius=256)
    


100%|██████████████████████████████████████████████████████████████████████████| 95/95 [01:32<00:00,  1.03it/s]


In [40]:
IDs=['109_A1c_ASAP_tumor_map', '10_A1a_ASAP_tumor_map', '10_A1b_ASAP_tumor_map', '10_A2b_ASAP_tumor_map', '110_A2b_ASAP_tumor_map', '112_a_ASAP_tumor_map', '112_b_ASAP_tumor_map', '123_A1a_ASAP_tumor_map', '12_A1c_ASAP_tumor_map', '14_A1b_ASAP_tumor_map', '14_A2b_ASAP_tumor_map', '169_A2b_ASAP_tumor_map', '270_A1b_ASAP_tumor_map', '270_A1d_ASAP_tumor_map', '270_A1e_ASAP_tumor_map', '270_A2b_ASAP_tumor_map', '270_A2f_ASAP_tumor_map', '281_A1d_ASAP_tumor_map', '281_A1f_ASAP_tumor_map', '281_A2eX_ASAP_tumor_map', '311_A2c_ASAP_tumor_map', '327_A1a_ASAP_tumor_map', '327_A1d_ASAP_tumor_map', '327_B1c_ASAP_tumor_map', '341_a_ASAP_tumor_map', '341_b_ASAP_tumor_map', '342_a_ASAP_tumor_map', '342_b_ASAP_tumor_map', '343_a_ASAP_tumor_map', '343_b_ASAP_tumor_map', '343_c_ASAP_tumor_map', '343_d_ASAP_tumor_map', '344_a_ASAP_tumor_map', '344_b_ASAP_tumor_map', '345_a_ASAP_tumor_map', '345_b_ASAP_tumor_map', '346_a_ASAP_tumor_map', '346_b_ASAP_tumor_map', '350_A1a_ASAP_tumor_map', '350_A1b_ASAP_tumor_map', '350_A1c_ASAP_tumor_map', '350_A1d_ASAP_tumor_map', '350_A1e_ASAP_tumor_map', '351_A2b_ASAP_tumor_map', '352_A1d_ASAP_tumor_map', '352_A1e_ASAP_tumor_map', '352_A1g_ASAP_tumor_map', '352_A1h_ASAP_tumor_map', '352_A1i_ASAP_tumor_map', '353_A2b_ASAP_tumor_map', '354_A1b_ASAP_tumor_map', '354_A1c_ASAP_tumor_map', '354_A1d_ASAP_tumor_map', '354_A3a_ASAP_tumor_map', '354_A3b_ASAP_tumor_map', '354_A3c_ASAP_tumor_map', '354_D1b_ASAP_tumor_map', '355_A1d_ASAP_tumor_map', '356_A1b_ASAP_tumor_map', '358_A1a_ASAP_tumor_map', '358_A1b_ASAP_tumor_map', '361_a_ASAP_tumor_map', '361_b_ASAP_tumor_map', '362_A1a_ASAP_tumor_map', '362_A1b_ASAP_tumor_map', '362_A1c_ASAP_tumor_map', '363_A1b_ASAP_tumor_map', '363_A1c_ASAP_tumor_map', '363_A2b_ASAP_tumor_map', '363_A3b_ASAP_tumor_map', '364_A1b_ASAP_tumor_map', '364_A2b_ASAP_tumor_map', '364_A4b_ASAP_tumor_map', '365_A1b_ASAP_tumor_map', '365_A2b_ASAP_tumor_map', '366_A1a_ASAP_tumor_map', '366_A1b_ASAP_tumor_map', '366_A1c_ASAP_tumor_map', '367_A2b_ASAP_tumor_map', '368_A1b_ASAP_tumor_map', '368_A1c_ASAP_tumor_map', '368_A1d_ASAP_tumor_map', '369_A1b_ASAP_tumor_map', '369_A1c_ASAP_tumor_map', '369_A2b_ASAP_tumor_map', '370_A1b_ASAP_tumor_map', '370_A2a_ASAP_tumor_map', '370_A2b_ASAP_tumor_map', '37_A2d_ASAP_tumor_map', '61_A1a_ASAP_tumor_map', '61_B1a_ASAP_tumor_map', '70_A2b_ASAP_tumor_map', '7_A1c_ASAP_tumor_map', '7_A1d_ASAP_tumor_map', '7_A1e_ASAP_tumor_map']

all_dataset=[]
for ID in (IDs):
    graph_dataset=get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/",
                  basename=ID +'.pkl',
                  target_col='scc',
                  k=8,
                  radius=256)
    all_dataset.append(graph_dataset)

[Data(x=[4639, 2048], edge_index=[2, 22765], y=[4639], pos=[4639, 2], train_mask=[4639], val_mask=[4639], test_mask=[4639]),
 Data(x=[4710, 2048], edge_index=[2, 23116], y=[4710], pos=[4710, 2], train_mask=[4710], val_mask=[4710], test_mask=[4710]),
 Data(x=[5083, 2048], edge_index=[2, 24985], y=[5083], pos=[5083, 2], train_mask=[5083], val_mask=[5083], test_mask=[5083])]

In [ ]:
import pickle
pickle.dump(dict(graph_dataset=graph_datasets,train_test_split))

In [13]:
df=pd.read_pickle("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/file_locations.pkl")

In [12]:
import math
from sklearn.model_selection import train_test_split
from itertools import chain
graph_datasets={}
for ID in tqdm.tqdm(IDs):
    all_dataset = get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/", 
                                    basename=ID+ ".pkl", 
                                    target_col='scc', 
                                    k=8, 
                                    radius=256, 
                                    dataset_save_loc=f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/{ID+'.pkl'}")                        
    print(ID, len(all_dataset))
    if len(all_dataset) == 1:
        graph_datasets[ID]= {'train' : all_dataset, 'val' : []}
    else:
        train_size=int(math.ceil(len(all_dataset)*0.9))
        train_dataset, val_dataset = train_test_split(all_dataset,train_size=0.90,random_state=42)
        graph_datasets[ID]= {'train' : train_dataset, 'val' : val_dataset}


  1%|▍                                          | 1/95 [00:02<04:03,  2.59s/it]

109_A1c_ASAP_tumor_map 2


  2%|▉                                          | 2/95 [00:03<02:30,  1.62s/it]

10_A1a_ASAP_tumor_map 3


  3%|█▎                                         | 3/95 [00:04<02:01,  1.32s/it]

...

7_A1c_ASAP_tumor_map 3


 99%|█████████████████████████████████████████▌| 94/95 [01:31<00:00,  1.21it/s]

7_A1d_ASAP_tumor_map 3


100%|██████████████████████████████████████████| 95/95 [01:31<00:00,  1.03it/s]

7_A1e_ASAP_tumor_map 3


In [13]:
graph_datasets

{'109_A1c_ASAP_tumor_map': {'train': [Data(x=[20589, 2048], edge_index=[2, 102219], y=[20589], pos=[20589, 2], train_mask=[20589], val_mask=[20589], test_mask=[20589])],
  'val': [Data(x=[20706, 2048], edge_index=[2, 102704], y=[20706], pos=[20706, 2], train_mask=[20706], val_mask=[20706], test_mask=[20706])]},
 '10_A1a_ASAP_tumor_map': {'train': [Data(x=[5563, 2048], edge_index=[2, 27309], y=[5563], pos=[5563, 2], train_mask=[5563], val_mask=[5563], test_mask=[5563]),
   Data(x=[5101, 2048], edge_index=[2, 24973], y=[5101], pos=[5101, 2], train_mask=[5101], val_mask=[5101], test_mask=[5101])],
  'val': [Data(x=[5804, 2048], edge_index=[2, 28532], y=[5804], pos=[5804, 2], train_mask=[5804], val_mask=[5804], test_mask=[5804])]},
 '10_A1b_ASAP_tumor_map': {'train': [Data(x=[5719, 2048], edge_index=[2, 28113], y=[5719], pos=[5719, 2], train_mask=[5719], val_mask=[5719], test_mask=[5719]),
   Data(x=[5649, 2048], edge_index=[2, 27761], y=[5649], pos=[5649, 2], train_mask=[5649], val_mask=[

In [1]:
graph_datasets={'109_A1c_ASAP_tumor_map': {'train': [Data(x=[20589, 2048], edge_index=[2, 102219], y=[20589], pos=[20589, 2], train_mask=[20589], val_mask=[20589], test_mask=[20589])], 'val': [Data(x=[20706, 2048], edge_index=[2, 102704], y=[20706], pos=[20706, 2], train_mask=[20706], val_mask=[20706], test_mask=[20706])]}, '10_A1a_ASAP_tumor_map':{'train': [Data(x=[5563, 2048], edge_index=[2, 27309], y=[5563], pos=[5563, 2], train_mask=[5563], val_mask=[5563], test_mask=[5563]), Data(x=[5101, 2048], edge_index=[2, 24973], y=[5101], pos=[5101, 2], train_mask=[5101], val_mask=[5101], test_mask=[5101])], 'val': [Data(x=[5804, 2048], edge_index=[2, 28532], y=[5804], pos=[5804, 2], train_mask=[5804], val_mask=[5804], test_mask=[5804])]}, '10_A1b_ASAP_tumor_map':{'train': [Data(x=[5719, 2048], edge_index=[2, 28113], y=[5719], pos=[5719, 2], train_mask=[5719], val_mask=[5719], test_mask=[5719]), Data(x=[5649, 2048], edge_index=[2, 27761], y=[5649], pos=[5649, 2], train_mask=[5649], val_mask=[5649], test_mask=[5649])], 'val': [Data(x=[5642, 2048], edge_index=[2, 27702], y=[5642], pos=[5642, 2], train_mask=[5642], val_mask=[5642], test_mask=[5642])]}, '10_A2b_ASAP_tumor_map':{'train': [Data(x=[6098, 2048], edge_index=[2, 29956], y=[6098], pos=[6098, 2], train_mask=[6098], val_mask=[6098], test_mask=[6098]), Data(x=[6239, 2048], edge_index=[2, 30715], y=[6239], pos=[6239, 2], train_mask=[6239], val_mask=[6239], test_mask=[6239])], 'val': [Data(x=[6085, 2048], edge_index=[2, 29929], y=[6085], pos=[6085, 2], train_mask=[6085], val_mask=[6085], test_mask=[6085])]}, '110_A2b_ASAP_tumor_map':{'train': [Data(x=[4220, 2048], edge_index=[2, 20574], y=[4220], pos=[4220, 2], train_mask=[4220], val_mask=[4220], test_mask=[4220]), Data(x=[4557, 2048], edge_index=[2, 22333], y=[4557], pos=[4557, 2], train_mask=[4557], val_mask=[4557], test_mask=[4557]), Data(x=[4607, 2048], edge_index=[2, 22561], y=[4607], pos=[4607, 2], train_mask=[4607], val_mask=[4607], test_mask=[4607])], 'val': [Data(x=[4472, 2048], edge_index=[2, 21880], y=[4472], pos=[4472, 2], train_mask=[4472], val_mask=[4472], test_mask=[4472])]}, '112_a_ASAP_tumor_map':{'train': [Data(x=[817, 2048], edge_index=[2, 3897], y=[817], pos=[817, 2], train_mask=[817], val_mask=[817], test_mask=[817]), Data(x=[795, 2048], edge_index=[2, 3785], y=[795], pos=[795, 2], train_mask=[795], val_mask=[795], test_mask=[795]), Data(x=[703, 2048], edge_index=[2, 3369], y=[703], pos=[703, 2], train_mask=[703], val_mask=[703], test_mask=[703]), Data(x=[832, 2048], edge_index=[2, 3972], y=[832], pos=[832, 2], train_mask=[832], val_mask=[832], test_mask=[832]), Data(x=[668, 2048], edge_index=[2, 3170], y=[668], pos=[668, 2], train_mask=[668], val_mask=[668], test_mask=[668]), Data(x=[884, 2048], edge_index=[2, 4244], y=[884], pos=[884, 2], train_mask=[884], val_mask=[884], test_mask=[884]), Data(x=[583, 2048], edge_index=[2, 2769], y=[583], pos=[583, 2], train_mask=[583], val_mask=[583], test_mask=[583])], 'val': [Data(x=[935, 2048], edge_index=[2, 4489], y=[935], pos=[935, 2], train_mask=[935], val_mask=[935], test_mask=[935])]}, '112_b_ASAP_tumor_map':{'train': [Data(x=[902, 2048], edge_index=[2, 4298], y=[902], pos=[902, 2], train_mask=[902], val_mask=[902], test_mask=[902]), Data(x=[815, 2048], edge_index=[2, 3889], y=[815], pos=[815, 2], train_mask=[815], val_mask=[815], test_mask=[815]), Data(x=[878, 2048], edge_index=[2, 4210], y=[878], pos=[878, 2], train_mask=[878], val_mask=[878], test_mask=[878]), Data(x=[844, 2048], edge_index=[2, 4004], y=[844], pos=[844, 2], train_mask=[844], val_mask=[844], test_mask=[844]), Data(x=[826, 2048], edge_index=[2, 3952], y=[826], pos=[826, 2], train_mask=[826], val_mask=[826], test_mask=[826]), Data(x=[871, 2048], edge_index=[2, 4135], y=[871], pos=[871, 2], train_mask=[871], val_mask=[871], test_mask=[871]), Data(x=[842, 2048], edge_index=[2, 4024], y=[842], pos=[842, 2], train_mask=[842], val_mask=[842], test_mask=[842])], 'val': [Data(x=[850, 2048], edge_index=[2, 4038], y=[850], pos=[850, 2], train_mask=[850], val_mask=[850], test_mask=[850])]}, '123_A1a_ASAP_tumor_map':{'train': [Data(x=[2626, 2048], edge_index=[2, 12492], y=[2626], pos=[2626, 2], train_mask=[2626], val_mask=[2626], test_mask=[2626]), Data(x=[1742, 2048], edge_index=[2, 8152], y=[1742], pos=[1742, 2], train_mask=[1742], val_mask=[1742], test_mask=[1742]), Data(x=[2955, 2048], edge_index=[2, 14357], y=[2955], pos=[2955, 2], train_mask=[2955], val_mask=[2955], test_mask=[2955]), Data(x=[2561, 2048], edge_index=[2, 12297], y=[2561], pos=[2561, 2], train_mask=[2561], val_mask=[2561], test_mask=[2561]), Data(x=[2825, 2048], edge_index=[2, 13425], y=[2825], pos=[2825, 2], train_mask=[2825], val_mask=[2825], test_mask=[2825])], 'val': [Data(x=[3280, 2048], edge_index=[2, 15970], y=[3280], pos=[3280, 2], train_mask=[3280], val_mask=[3280], test_mask=[3280])]}, '12_A1c_ASAP_tumor_map':{'train': [Data(x=[7126, 2048], edge_index=[2, 35066], y=[7126], pos=[7126, 2], train_mask=[7126], val_mask=[7126], test_mask=[7126]), Data(x=[6957, 2048], edge_index=[2, 34275], y=[6957], pos=[6957, 2], train_mask=[6957], val_mask=[6957], test_mask=[6957])], 'val': [Data(x=[6814, 2048], edge_index=[2, 33326], y=[6814], pos=[6814, 2], train_mask=[6814], val_mask=[6814], test_mask=[6814])]}, '14_A1b_ASAP_tumor_map':{'train': [Data(x=[5176, 2048], edge_index=[2, 25114], y=[5176], pos=[5176, 2], train_mask=[5176], val_mask=[5176], test_mask=[5176]), Data(x=[4841, 2048], edge_index=[2, 23269], y=[4841], pos=[4841, 2], train_mask=[4841], val_mask=[4841], test_mask=[4841])], 'val': [Data(x=[4577, 2048], edge_index=[2, 22143], y=[4577], pos=[4577, 2], train_mask=[4577], val_mask=[4577], test_mask=[4577])]}, '14_A2b_ASAP_tumor_map':{'train': [Data(x=[5075, 2048], edge_index=[2, 24045], y=[5075], pos=[5075, 2], train_mask=[5075], val_mask=[5075], test_mask=[5075]), Data(x=[4659, 2048], edge_index=[2, 22071], y=[4659], pos=[4659, 2], train_mask=[4659], val_mask=[4659], test_mask=[4659])], 'val': [Data(x=[5046, 2048], edge_index=[2, 24084], y=[5046], pos=[5046, 2], train_mask=[5046], val_mask=[5046], test_mask=[5046])]}, '169_A2b_ASAP_tumor_map':{'train': [Data(x=[5137, 2048], edge_index=[2, 25303], y=[5137], pos=[5137, 2], train_mask=[5137], val_mask=[5137], test_mask=[5137]), Data(x=[5193, 2048], edge_index=[2, 25555], y=[5193], pos=[5193, 2], train_mask=[5193], val_mask=[5193], test_mask=[5193])], 'val': [Data(x=[5220, 2048], edge_index=[2, 25698], y=[5220], pos=[5220, 2], train_mask=[5220], val_mask=[5220], test_mask=[5220])]}, '270_A1b_ASAP_tumor_map':{'train': [Data(x=[10733, 2048], edge_index=[2, 52753], y=[10733], pos=[10733, 2], train_mask=[10733], val_mask=[10733], test_mask=[10733])], 'val': [Data(x=[10638, 2048], edge_index=[2, 52354], y=[10638], pos=[10638, 2], train_mask=[10638], val_mask=[10638], test_mask=[10638])]}, '270_A1d_ASAP_tumor_map':{'train': [Data(x=[10457, 2048], edge_index=[2, 51557], y=[10457], pos=[10457, 2], train_mask=[10457], val_mask=[10457], test_mask=[10457])], 'val': [Data(x=[10268, 2048], edge_index=[2, 50640], y=[10268], pos=[10268, 2], train_mask=[10268], val_mask=[10268], test_mask=[10268])]}, '270_A1e_ASAP_tumor_map':{'train': [Data(x=[10168, 2048], edge_index=[2, 50196], y=[10168], pos=[10168, 2], train_mask=[10168], val_mask=[10168], test_mask=[10168])], 'val': [Data(x=[10423, 2048], edge_index=[2, 51437], y=[10423], pos=[10423, 2], train_mask=[10423], val_mask=[10423], test_mask=[10423])]}, '270_A2b_ASAP_tumor_map':{'train': [Data(x=[10205, 2048], edge_index=[2, 50345], y=[10205], pos=[10205, 2], train_mask=[10205], val_mask=[10205], test_mask=[10205])], 'val': [Data(x=[8924, 2048], edge_index=[2, 43966], y=[8924], pos=[8924, 2], train_mask=[8924], val_mask=[8924], test_mask=[8924])]}, '270_A2f_ASAP_tumor_map':{'train': [Data(x=[7420, 2048], edge_index=[2, 36228], y=[7420], pos=[7420, 2], train_mask=[7420], val_mask=[7420], test_mask=[7420])], 'val': []}, '281_A1d_ASAP_tumor_map':{'train': [Data(x=[6622, 2048], edge_index=[2, 32648], y=[6622], pos=[6622, 2], train_mask=[6622], val_mask=[6622], test_mask=[6622])], 'val': [Data(x=[6675, 2048], edge_index=[2, 32849], y=[6675], pos=[6675, 2], train_mask=[6675], val_mask=[6675], test_mask=[6675])]}, '281_A1f_ASAP_tumor_map':{'train': [Data(x=[7200, 2048], edge_index=[2, 35514], y=[7200], pos=[7200, 2], train_mask=[7200], val_mask=[7200], test_mask=[7200])], 'val': [Data(x=[7185, 2048], edge_index=[2, 35435], y=[7185], pos=[7185, 2], train_mask=[7185], val_mask=[7185], test_mask=[7185])]}, '281_A2eX_ASAP_tumor_map':{'train': [Data(x=[6892, 2048], edge_index=[2, 33970], y=[6892], pos=[6892, 2], train_mask=[6892], val_mask=[6892], test_mask=[6892])], 'val': [Data(x=[6557, 2048], edge_index=[2, 32353], y=[6557], pos=[6557, 2], train_mask=[6557], val_mask=[6557], test_mask=[6557])]}, '311_A2c_ASAP_tumor_map':{'train': [Data(x=[6043, 2048], edge_index=[2, 29801], y=[6043], pos=[6043, 2], train_mask=[6043], val_mask=[6043], test_mask=[6043])], 'val': [Data(x=[6294, 2048], edge_index=[2, 30852], y=[6294], pos=[6294, 2], train_mask=[6294], val_mask=[6294], test_mask=[6294])]}, '327_A1a_ASAP_tumor_map':{'train': [Data(x=[8925, 2048], edge_index=[2, 44005], y=[8925], pos=[8925, 2], train_mask=[8925], val_mask=[8925], test_mask=[8925]), Data(x=[7523, 2048], edge_index=[2, 36939], y=[7523], pos=[7523, 2], train_mask=[7523], val_mask=[7523], test_mask=[7523])], 'val': [Data(x=[9594, 2048], edge_index=[2, 47078], y=[9594], pos=[9594, 2], train_mask=[9594], val_mask=[9594], test_mask=[9594])]}, '327_A1d_ASAP_tumor_map':{'train': [Data(x=[8040, 2048], edge_index=[2, 39486], y=[8040], pos=[8040, 2], train_mask=[8040], val_mask=[8040], test_mask=[8040])], 'val': [Data(x=[8280, 2048], edge_index=[2, 40538], y=[8280], pos=[8280, 2], train_mask=[8280], val_mask=[8280], test_mask=[8280])]}, '327_B1c_ASAP_tumor_map':{'train': [Data(x=[18384, 2048], edge_index=[2, 90616], y=[18384], pos=[18384, 2], train_mask=[18384], val_mask=[18384], test_mask=[18384])], 'val': [Data(x=[20123, 2048], edge_index=[2, 99557], y=[20123], pos=[20123, 2], train_mask=[20123], val_mask=[20123], test_mask=[20123])]}, '341_a_ASAP_tumor_map':{'train': [Data(x=[3285, 2048], edge_index=[2, 16061], y=[3285], pos=[3285, 2], train_mask=[3285], val_mask=[3285], test_mask=[3285]), Data(x=[3282, 2048], edge_index=[2, 16056], y=[3282], pos=[3282, 2], train_mask=[3282], val_mask=[3282], test_mask=[3282])], 'val': [Data(x=[2812, 2048], edge_index=[2, 13740], y=[2812], pos=[2812, 2], train_mask=[2812], val_mask=[2812], test_mask=[2812])]}, '341_b_ASAP_tumor_map':{'train': [Data(x=[2217, 2048], edge_index=[2, 10801], y=[2217], pos=[2217, 2], train_mask=[2217], val_mask=[2217], test_mask=[2217]), Data(x=[2806, 2048], edge_index=[2, 13728], y=[2806], pos=[2806, 2], train_mask=[2806], val_mask=[2806], test_mask=[2806])], 'val': [Data(x=[1747, 2048], edge_index=[2, 8461], y=[1747], pos=[1747, 2], train_mask=[1747], val_mask=[1747], test_mask=[1747])]}, '342_a_ASAP_tumor_map':{'train': [Data(x=[817, 2048], edge_index=[2, 3897], y=[817], pos=[817, 2], train_mask=[817], val_mask=[817], test_mask=[817]), Data(x=[795, 2048], edge_index=[2, 3785], y=[795], pos=[795, 2], train_mask=[795], val_mask=[795], test_mask=[795]), Data(x=[703, 2048], edge_index=[2, 3369], y=[703], pos=[703, 2], train_mask=[703], val_mask=[703], test_mask=[703]), Data(x=[832, 2048], edge_index=[2, 3972], y=[832], pos=[832, 2], train_mask=[832], val_mask=[832], test_mask=[832]), Data(x=[668, 2048], edge_index=[2, 3170], y=[668], pos=[668, 2], train_mask=[668], val_mask=[668], test_mask=[668]), Data(x=[884, 2048], edge_index=[2, 4244], y=[884], pos=[884, 2], train_mask=[884], val_mask=[884], test_mask=[884]), Data(x=[583, 2048], edge_index=[2, 2769], y=[583], pos=[583, 2], train_mask=[583], val_mask=[583], test_mask=[583])], 'val': [Data(x=[935, 2048], edge_index=[2, 4489], y=[935], pos=[935, 2], train_mask=[935], val_mask=[935], test_mask=[935])]}, '342_b_ASAP_tumor_map':{'train': [Data(x=[902, 2048], edge_index=[2, 4298], y=[902], pos=[902, 2], train_mask=[902], val_mask=[902], test_mask=[902]), Data(x=[815, 2048], edge_index=[2, 3889], y=[815], pos=[815, 2], train_mask=[815], val_mask=[815], test_mask=[815]), Data(x=[878, 2048], edge_index=[2, 4210], y=[878], pos=[878, 2], train_mask=[878], val_mask=[878], test_mask=[878]), Data(x=[844, 2048], edge_index=[2, 4004], y=[844], pos=[844, 2], train_mask=[844], val_mask=[844], test_mask=[844]), Data(x=[826, 2048], edge_index=[2, 3952], y=[826], pos=[826, 2], train_mask=[826], val_mask=[826], test_mask=[826]), Data(x=[871, 2048], edge_index=[2, 4135], y=[871], pos=[871, 2], train_mask=[871], val_mask=[871], test_mask=[871]), Data(x=[842, 2048], edge_index=[2, 4024], y=[842], pos=[842, 2], train_mask=[842], val_mask=[842], test_mask=[842])], 'val': [Data(x=[850, 2048], edge_index=[2, 4038], y=[850], pos=[850, 2], train_mask=[850], val_mask=[850], test_mask=[850])]}, '343_a_ASAP_tumor_map':{'train': [Data(x=[6056, 2048], edge_index=[2, 28736], y=[6056], pos=[6056, 2], train_mask=[6056], val_mask=[6056], test_mask=[6056]), Data(x=[217, 2048], edge_index=[2, 991], y=[217], pos=[217, 2], train_mask=[217], val_mask=[217], test_mask=[217])], 'val': [Data(x=[10754, 2048], edge_index=[2, 51502], y=[10754], pos=[10754, 2], train_mask=[10754], val_mask=[10754], test_mask=[10754])]}, '343_b_ASAP_tumor_map':{'train': [Data(x=[22341, 2048], edge_index=[2, 109241], y=[22341], pos=[22341, 2], train_mask=[22341], val_mask=[22341], test_mask=[22341])], 'val': []}, '343_c_ASAP_tumor_map':{'train': [Data(x=[8943, 2048], edge_index=[2, 43861], y=[8943], pos=[8943, 2], train_mask=[8943], val_mask=[8943], test_mask=[8943])], 'val': [Data(x=[14272, 2048], edge_index=[2, 69508], y=[14272], pos=[14272, 2], train_mask=[14272], val_mask=[14272], test_mask=[14272])]}, '343_d_ASAP_tumor_map':{'train': [Data(x=[23840, 2048], edge_index=[2, 116080], y=[23840], pos=[23840, 2], train_mask=[23840], val_mask=[23840], test_mask=[23840])], 'val': []}, '344_a_ASAP_tumor_map':{'train': [Data(x=[2319, 2048], edge_index=[2, 11133], y=[2319], pos=[2319, 2], train_mask=[2319], val_mask=[2319], test_mask=[2319]), Data(x=[2536, 2048], edge_index=[2, 12266], y=[2536], pos=[2536, 2], train_mask=[2536], val_mask=[2536], test_mask=[2536]), Data(x=[2706, 2048], edge_index=[2, 13020], y=[2706], pos=[2706, 2], train_mask=[2706], val_mask=[2706], test_mask=[2706])], 'val': [Data(x=[2423, 2048], edge_index=[2, 11553], y=[2423], pos=[2423, 2], train_mask=[2423], val_mask=[2423], test_mask=[2423])]}, '344_b_ASAP_tumor_map':{'train': [Data(x=[2642, 2048], edge_index=[2, 12780], y=[2642], pos=[2642, 2], train_mask=[2642], val_mask=[2642], test_mask=[2642]), Data(x=[2598, 2048], edge_index=[2, 12504], y=[2598], pos=[2598, 2], train_mask=[2598], val_mask=[2598], test_mask=[2598]), Data(x=[2578, 2048], edge_index=[2, 12512], y=[2578], pos=[2578, 2], train_mask=[2578], val_mask=[2578], test_mask=[2578])], 'val': [Data(x=[2523, 2048], edge_index=[2, 12237], y=[2523], pos=[2523, 2], train_mask=[2523], val_mask=[2523], test_mask=[2523])]}, '345_a_ASAP_tumor_map':{'train': [Data(x=[5059, 2048], edge_index=[2, 24891], y=[5059], pos=[5059, 2], train_mask=[5059], val_mask=[5059], test_mask=[5059]), Data(x=[4642, 2048], edge_index=[2, 22826], y=[4642], pos=[4642, 2], train_mask=[4642], val_mask=[4642], test_mask=[4642])], 'val': [Data(x=[4837, 2048], edge_index=[2, 23785], y=[4837], pos=[4837, 2], train_mask=[4837], val_mask=[4837], test_mask=[4837])]}, '345_b_ASAP_tumor_map':{'train': [Data(x=[5019, 2048], edge_index=[2, 24651], y=[5019], pos=[5019, 2], train_mask=[5019], val_mask=[5019], test_mask=[5019]), Data(x=[5072, 2048], edge_index=[2, 24926], y=[5072], pos=[5072, 2], train_mask=[5072], val_mask=[5072], test_mask=[5072])], 'val': [Data(x=[4723, 2048], edge_index=[2, 23209], y=[4723], pos=[4723, 2], train_mask=[4723], val_mask=[4723], test_mask=[4723])]}, '346_a_ASAP_tumor_map':{'train': [Data(x=[2608, 2048], edge_index=[2, 12462], y=[2608], pos=[2608, 2], train_mask=[2608], val_mask=[2608], test_mask=[2608]), Data(x=[532, 2048], edge_index=[2, 2502], y=[532], pos=[532, 2], train_mask=[532], val_mask=[532], test_mask=[532]), Data(x=[2529, 2048], edge_index=[2, 12003], y=[2529], pos=[2529, 2], train_mask=[2529], val_mask=[2529], test_mask=[2529]), Data(x=[2207, 2048], edge_index=[2, 10509], y=[2207], pos=[2207, 2], train_mask=[2207], val_mask=[2207], test_mask=[2207]), Data(x=[2774, 2048], edge_index=[2, 13196], y=[2774], pos=[2774, 2], train_mask=[2774], val_mask=[2774], test_mask=[2774])], 'val': [Data(x=[296, 2048], edge_index=[2, 1326], y=[296], pos=[296, 2], train_mask=[296], val_mask=[296], test_mask=[296])]}, '346_b_ASAP_tumor_map':{'train': [Data(x=[2110, 2048], edge_index=[2, 10048], y=[2110], pos=[2110, 2], train_mask=[2110], val_mask=[2110], test_mask=[2110]), Data(x=[256, 2048], edge_index=[2, 1190], y=[256], pos=[256, 2], train_mask=[256], val_mask=[256], test_mask=[256]), Data(x=[647, 2048], edge_index=[2, 3041], y=[647], pos=[647, 2], train_mask=[647], val_mask=[647], test_mask=[647]), Data(x=[1926, 2048], edge_index=[2, 9266], y=[1926], pos=[1926, 2], train_mask=[1926], val_mask=[1926], test_mask=[1926]), Data(x=[256, 2048], edge_index=[2, 1138], y=[256], pos=[256, 2], train_mask=[256], val_mask=[256], test_mask=[256]), Data(x=[1968, 2048], edge_index=[2, 9308], y=[1968], pos=[1968, 2], train_mask=[1968], val_mask=[1968], test_mask=[1968]), Data(x=[2800, 2048], edge_index=[2, 13298], y=[2800], pos=[2800, 2], train_mask=[2800], val_mask=[2800], test_mask=[2800]), Data(x=[369, 2048], edge_index=[2, 1685], y=[369], pos=[369, 2], train_mask=[369], val_mask=[369], test_mask=[369]), Data(x=[358, 2048], edge_index=[2, 1636], y=[358], pos=[358, 2], train_mask=[358], val_mask=[358], test_mask=[358])], 'val': [Data(x=[223, 2048], edge_index=[2, 1035], y=[223], pos=[223, 2], train_mask=[223], val_mask=[223], test_mask=[223]), Data(x=[375, 2048], edge_index=[2, 1731], y=[375], pos=[375, 2], train_mask=[375], val_mask=[375], test_mask=[375])]}, '350_A1a_ASAP_tumor_map':{'train': [Data(x=[2152, 2048], edge_index=[2, 10320], y=[2152], pos=[2152, 2], train_mask=[2152], val_mask=[2152], test_mask=[2152]), Data(x=[2573, 2048], edge_index=[2, 12497], y=[2573], pos=[2573, 2], train_mask=[2573], val_mask=[2573], test_mask=[2573]), Data(x=[5921, 2048], edge_index=[2, 28809], y=[5921], pos=[5921, 2], train_mask=[5921], val_mask=[5921], test_mask=[5921]), Data(x=[2435, 2048], edge_index=[2, 11667], y=[2435], pos=[2435, 2], train_mask=[2435], val_mask=[2435], test_mask=[2435])], 'val': [Data(x=[2947, 2048], edge_index=[2, 14323], y=[2947], pos=[2947, 2], train_mask=[2947], val_mask=[2947], test_mask=[2947])]}, '350_A1b_ASAP_tumor_map':{'train': [Data(x=[6168, 2048], edge_index=[2, 30254], y=[6168], pos=[6168, 2], train_mask=[6168], val_mask=[6168], test_mask=[6168]), Data(x=[6601, 2048], edge_index=[2, 32313], y=[6601], pos=[6601, 2], train_mask=[6601], val_mask=[6601], test_mask=[6601])], 'val': [Data(x=[6628, 2048], edge_index=[2, 32542], y=[6628], pos=[6628, 2], train_mask=[6628], val_mask=[6628], test_mask=[6628])]}, '350_A1c_ASAP_tumor_map':{'train': [Data(x=[6353, 2048], edge_index=[2, 31203], y=[6353], pos=[6353, 2], train_mask=[6353], val_mask=[6353], test_mask=[6353]), Data(x=[6222, 2048], edge_index=[2, 30556], y=[6222], pos=[6222, 2], train_mask=[6222], val_mask=[6222], test_mask=[6222])], 'val': [Data(x=[5921, 2048], edge_index=[2, 29101], y=[5921], pos=[5921, 2], train_mask=[5921], val_mask=[5921], test_mask=[5921])]}, '350_A1d_ASAP_tumor_map':{'train': [Data(x=[5494, 2048], edge_index=[2, 26986], y=[5494], pos=[5494, 2], train_mask=[5494], val_mask=[5494], test_mask=[5494]), Data(x=[5831, 2048], edge_index=[2, 28669], y=[5831], pos=[5831, 2], train_mask=[5831], val_mask=[5831], test_mask=[5831])], 'val': [Data(x=[5516, 2048], edge_index=[2, 27058], y=[5516], pos=[5516, 2], train_mask=[5516], val_mask=[5516], test_mask=[5516])]}, '350_A1e_ASAP_tumor_map':{'train': [Data(x=[4710, 2048], edge_index=[2, 23116], y=[4710], pos=[4710, 2], train_mask=[4710], val_mask=[4710], test_mask=[4710]), Data(x=[5083, 2048], edge_index=[2, 24985], y=[5083], pos=[5083, 2], train_mask=[5083], val_mask=[5083], test_mask=[5083])], 'val': [Data(x=[4639, 2048], edge_index=[2, 22765], y=[4639], pos=[4639, 2], train_mask=[4639], val_mask=[4639], test_mask=[4639])]}, '351_A2b_ASAP_tumor_map':{'train': [Data(x=[1686, 2048], edge_index=[2, 8148], y=[1686], pos=[1686, 2], train_mask=[1686], val_mask=[1686], test_mask=[1686]), Data(x=[1663, 2048], edge_index=[2, 8025], y=[1663], pos=[1663, 2], train_mask=[1663], val_mask=[1663], test_mask=[1663]), Data(x=[1787, 2048], edge_index=[2, 8725], y=[1787], pos=[1787, 2], train_mask=[1787], val_mask=[1787], test_mask=[1787]), Data(x=[1805, 2048], edge_index=[2, 8797], y=[1805], pos=[1805, 2], train_mask=[1805], val_mask=[1805], test_mask=[1805]), Data(x=[1665, 2048], edge_index=[2, 8099], y=[1665], pos=[1665, 2], train_mask=[1665], val_mask=[1665], test_mask=[1665])], 'val': [Data(x=[1672, 2048], edge_index=[2, 8158], y=[1672], pos=[1672, 2], train_mask=[1672], val_mask=[1672], test_mask=[1672])]}, '352_A1d_ASAP_tumor_map':{'train': [Data(x=[3270, 2048], edge_index=[2, 16022], y=[3270], pos=[3270, 2], train_mask=[3270], val_mask=[3270], test_mask=[3270]), Data(x=[3424, 2048], edge_index=[2, 16834], y=[3424], pos=[3424, 2], train_mask=[3424], val_mask=[3424], test_mask=[3424]), Data(x=[3966, 2048], edge_index=[2, 19470], y=[3966], pos=[3966, 2], train_mask=[3966], val_mask=[3966], test_mask=[3966]), Data(x=[3803, 2048], edge_index=[2, 18603], y=[3803], pos=[3803, 2], train_mask=[3803], val_mask=[3803], test_mask=[3803]), Data(x=[3460, 2048], edge_index=[2, 16982], y=[3460], pos=[3460, 2], train_mask=[3460], val_mask=[3460], test_mask=[3460])], 'val': [Data(x=[3325, 2048], edge_index=[2, 16313], y=[3325], pos=[3325, 2], train_mask=[3325], val_mask=[3325], test_mask=[3325])]}, '352_A1e_ASAP_tumor_map':{'train': [Data(x=[3076, 2048], edge_index=[2, 15040], y=[3076], pos=[3076, 2], train_mask=[3076], val_mask=[3076], test_mask=[3076]), Data(x=[3317, 2048], edge_index=[2, 16267], y=[3317], pos=[3317, 2], train_mask=[3317], val_mask=[3317], test_mask=[3317]), Data(x=[3746, 2048], edge_index=[2, 18414], y=[3746], pos=[3746, 2], train_mask=[3746], val_mask=[3746], test_mask=[3746]), Data(x=[3797, 2048], edge_index=[2, 18657], y=[3797], pos=[3797, 2], train_mask=[3797], val_mask=[3797], test_mask=[3797]), Data(x=[3216, 2048], edge_index=[2, 15736], y=[3216], pos=[3216, 2], train_mask=[3216], val_mask=[3216], test_mask=[3216])], 'val': [Data(x=[3533, 2048], edge_index=[2, 17355], y=[3533], pos=[3533, 2], train_mask=[3533], val_mask=[3533], test_mask=[3533])]}, '352_A1g_ASAP_tumor_map':{'train': [Data(x=[2713, 2048], edge_index=[2, 13289], y=[2713], pos=[2713, 2], train_mask=[2713], val_mask=[2713], test_mask=[2713]), Data(x=[2708, 2048], edge_index=[2, 13252], y=[2708], pos=[2708, 2], train_mask=[2708], val_mask=[2708], test_mask=[2708]), Data(x=[2900, 2048], edge_index=[2, 14226], y=[2900], pos=[2900, 2], train_mask=[2900], val_mask=[2900], test_mask=[2900]), Data(x=[2921, 2048], edge_index=[2, 14269], y=[2921], pos=[2921, 2], train_mask=[2921], val_mask=[2921], test_mask=[2921]), Data(x=[2752, 2048], edge_index=[2, 13480], y=[2752], pos=[2752, 2], train_mask=[2752], val_mask=[2752], test_mask=[2752])], 'val': [Data(x=[3023, 2048], edge_index=[2, 14817], y=[3023], pos=[3023, 2], train_mask=[3023], val_mask=[3023], test_mask=[3023])]}, '352_A1h_ASAP_tumor_map':{'train': [Data(x=[2578, 2048], edge_index=[2, 12626], y=[2578], pos=[2578, 2], train_mask=[2578], val_mask=[2578], test_mask=[2578]), Data(x=[2365, 2048], edge_index=[2, 11465], y=[2365], pos=[2365, 2], train_mask=[2365], val_mask=[2365], test_mask=[2365]), Data(x=[2314, 2048], edge_index=[2, 11216], y=[2314], pos=[2314, 2], train_mask=[2314], val_mask=[2314], test_mask=[2314]), Data(x=[2572, 2048], edge_index=[2, 12592], y=[2572], pos=[2572, 2], train_mask=[2572], val_mask=[2572], test_mask=[2572]), Data(x=[2198, 2048], edge_index=[2, 10612], y=[2198], pos=[2198, 2], train_mask=[2198], val_mask=[2198], test_mask=[2198])], 'val': [Data(x=[2780, 2048], edge_index=[2, 13618], y=[2780], pos=[2780, 2], train_mask=[2780], val_mask=[2780], test_mask=[2780])]}, '352_A1i_ASAP_tumor_map':{'train': [Data(x=[2440, 2048], edge_index=[2, 11852], y=[2440], pos=[2440, 2], train_mask=[2440], val_mask=[2440], test_mask=[2440]), Data(x=[2460, 2048], edge_index=[2, 12036], y=[2460], pos=[2460, 2], train_mask=[2460], val_mask=[2460], test_mask=[2460]), Data(x=[2678, 2048], edge_index=[2, 13116], y=[2678], pos=[2678, 2], train_mask=[2678], val_mask=[2678], test_mask=[2678]), Data(x=[2647, 2048], edge_index=[2, 12957], y=[2647], pos=[2647, 2], train_mask=[2647], val_mask=[2647], test_mask=[2647]), Data(x=[2462, 2048], edge_index=[2, 12048], y=[2462], pos=[2462, 2], train_mask=[2462], val_mask=[2462], test_mask=[2462])], 'val': [Data(x=[1890, 2048], edge_index=[2, 9174], y=[1890], pos=[1890, 2], train_mask=[1890], val_mask=[1890], test_mask=[1890])]}, '353_A2b_ASAP_tumor_map':{'train': [Data(x=[4220, 2048], edge_index=[2, 20574], y=[4220], pos=[4220, 2], train_mask=[4220], val_mask=[4220], test_mask=[4220]), Data(x=[4557, 2048], edge_index=[2, 22333], y=[4557], pos=[4557, 2], train_mask=[4557], val_mask=[4557], test_mask=[4557]), Data(x=[4607, 2048], edge_index=[2, 22561], y=[4607], pos=[4607, 2], train_mask=[4607], val_mask=[4607], test_mask=[4607])], 'val': [Data(x=[4472, 2048], edge_index=[2, 21880], y=[4472], pos=[4472, 2], train_mask=[4472], val_mask=[4472], test_mask=[4472])]}, '354_A1b_ASAP_tumor_map':{'train': [Data(x=[13521, 2048], edge_index=[2, 65591], y=[13521], pos=[13521, 2], train_mask=[13521], val_mask=[13521], test_mask=[13521])], 'val': []}, '354_A1c_ASAP_tumor_map':{'train': [Data(x=[15681, 2048], edge_index=[2, 76283], y=[15681], pos=[15681, 2], train_mask=[15681], val_mask=[15681], test_mask=[15681])], 'val': []}, '354_A1d_ASAP_tumor_map':{'train': [Data(x=[18198, 2048], edge_index=[2, 89710], y=[18198], pos=[18198, 2], train_mask=[18198], val_mask=[18198], test_mask=[18198])], 'val': []}, '354_A3a_ASAP_tumor_map':{'train': [Data(x=[459, 2048], edge_index=[2, 2023], y=[459], pos=[459, 2], train_mask=[459], val_mask=[459], test_mask=[459]), Data(x=[8477, 2048], edge_index=[2, 40317], y=[8477], pos=[8477, 2], train_mask=[8477], val_mask=[8477], test_mask=[8477]), Data(x=[2248, 2048], edge_index=[2, 10640], y=[2248], pos=[2248, 2], train_mask=[2248], val_mask=[2248], test_mask=[2248])], 'val': [Data(x=[2063, 2048], edge_index=[2, 9639], y=[2063], pos=[2063, 2], train_mask=[2063], val_mask=[2063], test_mask=[2063])]}, '354_A3b_ASAP_tumor_map':{'train': [Data(x=[11270, 2048], edge_index=[2, 54620], y=[11270], pos=[11270, 2], train_mask=[11270], val_mask=[11270], test_mask=[11270])], 'val': [Data(x=[10642, 2048], edge_index=[2, 51608], y=[10642], pos=[10642, 2], train_mask=[10642], val_mask=[10642], test_mask=[10642])]}, '354_A3c_ASAP_tumor_map':{'train': [Data(x=[12102, 2048], edge_index=[2, 58352], y=[12102], pos=[12102, 2], train_mask=[12102], val_mask=[12102], test_mask=[12102])], 'val': [Data(x=[13586, 2048], edge_index=[2, 66282], y=[13586], pos=[13586, 2], train_mask=[13586], val_mask=[13586], test_mask=[13586])]}, '354_D1b_ASAP_tumor_map':{'train': [Data(x=[7910, 2048], edge_index=[2, 38174], y=[7910], pos=[7910, 2], train_mask=[7910], val_mask=[7910], test_mask=[7910])], 'val': [Data(x=[7125, 2048], edge_index=[2, 34687], y=[7125], pos=[7125, 2], train_mask=[7125], val_mask=[7125], test_mask=[7125])]}, '355_A1d_ASAP_tumor_map':{'train': [Data(x=[2793, 2048], edge_index=[2, 13597], y=[2793], pos=[2793, 2], train_mask=[2793], val_mask=[2793], test_mask=[2793]), Data(x=[2950, 2048], edge_index=[2, 14440], y=[2950], pos=[2950, 2], train_mask=[2950], val_mask=[2950], test_mask=[2950])], 'val': [Data(x=[2684, 2048], edge_index=[2, 13052], y=[2684], pos=[2684, 2], train_mask=[2684], val_mask=[2684], test_mask=[2684])]}, '356_A1b_ASAP_tumor_map':{'train': [Data(x=[3738, 2048], edge_index=[2, 18284], y=[3738], pos=[3738, 2], train_mask=[3738], val_mask=[3738], test_mask=[3738]), Data(x=[3631, 2048], edge_index=[2, 17755], y=[3631], pos=[3631, 2], train_mask=[3631], val_mask=[3631], test_mask=[3631])], 'val': [Data(x=[3395, 2048], edge_index=[2, 16605], y=[3395], pos=[3395, 2], train_mask=[3395], val_mask=[3395], test_mask=[3395])]}, '358_A1a_ASAP_tumor_map':{'train': [Data(x=[2626, 2048], edge_index=[2, 12492], y=[2626], pos=[2626, 2], train_mask=[2626], val_mask=[2626], test_mask=[2626]), Data(x=[1742, 2048], edge_index=[2, 8152], y=[1742], pos=[1742, 2], train_mask=[1742], val_mask=[1742], test_mask=[1742]), Data(x=[2955, 2048], edge_index=[2, 14357], y=[2955], pos=[2955, 2], train_mask=[2955], val_mask=[2955], test_mask=[2955]), Data(x=[2561, 2048], edge_index=[2, 12297], y=[2561], pos=[2561, 2], train_mask=[2561], val_mask=[2561], test_mask=[2561]), Data(x=[2825, 2048], edge_index=[2, 13425], y=[2825], pos=[2825, 2], train_mask=[2825], val_mask=[2825], test_mask=[2825])], 'val': [Data(x=[3280, 2048], edge_index=[2, 15970], y=[3280], pos=[3280, 2], train_mask=[3280], val_mask=[3280], test_mask=[3280])]}, '358_A1b_ASAP_tumor_map':{'train': [Data(x=[3117, 2048], edge_index=[2, 15001], y=[3117], pos=[3117, 2], train_mask=[3117], val_mask=[3117], test_mask=[3117]), Data(x=[2663, 2048], edge_index=[2, 12951], y=[2663], pos=[2663, 2], train_mask=[2663], val_mask=[2663], test_mask=[2663]), Data(x=[5892, 2048], edge_index=[2, 28576], y=[5892], pos=[5892, 2], train_mask=[5892], val_mask=[5892], test_mask=[5892]), Data(x=[3544, 2048], edge_index=[2, 17260], y=[3544], pos=[3544, 2], train_mask=[3544], val_mask=[3544], test_mask=[3544])], 'val': [Data(x=[3285, 2048], edge_index=[2, 15963], y=[3285], pos=[3285, 2], train_mask=[3285], val_mask=[3285], test_mask=[3285])]}, '361_a_ASAP_tumor_map':{'train': [Data(x=[5045, 2048], edge_index=[2, 24627], y=[5045], pos=[5045, 2], train_mask=[5045], val_mask=[5045], test_mask=[5045]), Data(x=[506, 2048], edge_index=[2, 2386], y=[506], pos=[506, 2], train_mask=[506], val_mask=[506], test_mask=[506]), Data(x=[8312, 2048], edge_index=[2, 40554], y=[8312], pos=[8312, 2], train_mask=[8312], val_mask=[8312], test_mask=[8312])], 'val': [Data(x=[2890, 2048], edge_index=[2, 14014], y=[2890], pos=[2890, 2], train_mask=[2890], val_mask=[2890], test_mask=[2890])]}, '361_b_ASAP_tumor_map':{'train': [Data(x=[15269, 2048], edge_index=[2, 74929], y=[15269], pos=[15269, 2], train_mask=[15269], val_mask=[15269], test_mask=[15269])], 'val': [Data(x=[6360, 2048], edge_index=[2, 31250], y=[6360], pos=[6360, 2], train_mask=[6360], val_mask=[6360], test_mask=[6360])]}, '362_A1a_ASAP_tumor_map':{'train': [Data(x=[10656, 2048], edge_index=[2, 52170], y=[10656], pos=[10656, 2], train_mask=[10656], val_mask=[10656], test_mask=[10656])], 'val': [Data(x=[8954, 2048], edge_index=[2, 43558], y=[8954], pos=[8954, 2], train_mask=[8954], val_mask=[8954], test_mask=[8954])]}, '362_A1b_ASAP_tumor_map':{'train': [Data(x=[10277, 2048], edge_index=[2, 50425], y=[10277], pos=[10277, 2], train_mask=[10277], val_mask=[10277], test_mask=[10277])], 'val': [Data(x=[10950, 2048], edge_index=[2, 53728], y=[10950], pos=[10950, 2], train_mask=[10950], val_mask=[10950], test_mask=[10950])]}, '362_A1c_ASAP_tumor_map':{'train': [Data(x=[8771, 2048], edge_index=[2, 42925], y=[8771], pos=[8771, 2], train_mask=[8771], val_mask=[8771], test_mask=[8771])], 'val': [Data(x=[10025, 2048], edge_index=[2, 49095], y=[10025], pos=[10025, 2], train_mask=[10025], val_mask=[10025], test_mask=[10025])]}, '363_A1b_ASAP_tumor_map':{'train': [Data(x=[10713, 2048], edge_index=[2, 52717], y=[10713], pos=[10713, 2], train_mask=[10713], val_mask=[10713], test_mask=[10713])], 'val': [Data(x=[10069, 2048], edge_index=[2, 49471], y=[10069], pos=[10069, 2], train_mask=[10069], val_mask=[10069], test_mask=[10069])]}, '363_A1c_ASAP_tumor_map':{'train': [Data(x=[10333, 2048], edge_index=[2, 50949], y=[10333], pos=[10333, 2], train_mask=[10333], val_mask=[10333], test_mask=[10333])], 'val': [Data(x=[10265, 2048], edge_index=[2, 50613], y=[10265], pos=[10265, 2], train_mask=[10265], val_mask=[10265], test_mask=[10265])]}, '363_A2b_ASAP_tumor_map':{'train': [Data(x=[8385, 2048], edge_index=[2, 41225], y=[8385], pos=[8385, 2], train_mask=[8385], val_mask=[8385], test_mask=[8385]), Data(x=[8318, 2048], edge_index=[2, 40800], y=[8318], pos=[8318, 2], train_mask=[8318], val_mask=[8318], test_mask=[8318])], 'val': [Data(x=[8570, 2048], edge_index=[2, 42180], y=[8570], pos=[8570, 2], train_mask=[8570], val_mask=[8570], test_mask=[8570])]}, '363_A3b_ASAP_tumor_map':{'train': [Data(x=[11918, 2048], edge_index=[2, 58766], y=[11918], pos=[11918, 2], train_mask=[11918], val_mask=[11918], test_mask=[11918])], 'val': [Data(x=[11553, 2048], edge_index=[2, 56695], y=[11553], pos=[11553, 2], train_mask=[11553], val_mask=[11553], test_mask=[11553])]}, '364_A1b_ASAP_tumor_map':{'train': [Data(x=[7890, 2048], edge_index=[2, 38608], y=[7890], pos=[7890, 2], train_mask=[7890], val_mask=[7890], test_mask=[7890]), Data(x=[7228, 2048], edge_index=[2, 35234], y=[7228], pos=[7228, 2], train_mask=[7228], val_mask=[7228], test_mask=[7228])], 'val': [Data(x=[7129, 2048], edge_index=[2, 34695], y=[7129], pos=[7129, 2], train_mask=[7129], val_mask=[7129], test_mask=[7129])]}, '364_A2b_ASAP_tumor_map':{'train': [Data(x=[7762, 2048], edge_index=[2, 37502], y=[7762], pos=[7762, 2], train_mask=[7762], val_mask=[7762], test_mask=[7762]), Data(x=[6950, 2048], edge_index=[2, 33088], y=[6950], pos=[6950, 2], train_mask=[6950], val_mask=[6950], test_mask=[6950])], 'val': [Data(x=[7887, 2048], edge_index=[2, 38045], y=[7887], pos=[7887, 2], train_mask=[7887], val_mask=[7887], test_mask=[7887])]}, '364_A4b_ASAP_tumor_map':{'train': [Data(x=[7567, 2048], edge_index=[2, 36695], y=[7567], pos=[7567, 2], train_mask=[7567], val_mask=[7567], test_mask=[7567]), Data(x=[6416, 2048], edge_index=[2, 30756], y=[6416], pos=[6416, 2], train_mask=[6416], val_mask=[6416], test_mask=[6416])], 'val': [Data(x=[8013, 2048], edge_index=[2, 38809], y=[8013], pos=[8013, 2], train_mask=[8013], val_mask=[8013], test_mask=[8013])]}, '365_A1b_ASAP_tumor_map':{'train': [Data(x=[6347, 2048], edge_index=[2, 31109], y=[6347], pos=[6347, 2], train_mask=[6347], val_mask=[6347], test_mask=[6347]), Data(x=[186, 2048], edge_index=[2, 812], y=[186], pos=[186, 2], train_mask=[186], val_mask=[186], test_mask=[186]), Data(x=[6236, 2048], edge_index=[2, 30544], y=[6236], pos=[6236, 2], train_mask=[6236], val_mask=[6236], test_mask=[6236])], 'val': [Data(x=[6763, 2048], edge_index=[2, 33205], y=[6763], pos=[6763, 2], train_mask=[6763], val_mask=[6763], test_mask=[6763])]}, '365_A2b_ASAP_tumor_map':{'train': [Data(x=[6896, 2048], edge_index=[2, 33746], y=[6896], pos=[6896, 2], train_mask=[6896], val_mask=[6896], test_mask=[6896]), Data(x=[5952, 2048], edge_index=[2, 29198], y=[5952], pos=[5952, 2], train_mask=[5952], val_mask=[5952], test_mask=[5952])], 'val': [Data(x=[5449, 2048], edge_index=[2, 26585], y=[5449], pos=[5449, 2], train_mask=[5449], val_mask=[5449], test_mask=[5449])]}, '366_A1a_ASAP_tumor_map':{'train': [Data(x=[7163, 2048], edge_index=[2, 34633], y=[7163], pos=[7163, 2], train_mask=[7163], val_mask=[7163], test_mask=[7163])], 'val': [Data(x=[6673, 2048], edge_index=[2, 32347], y=[6673], pos=[6673, 2], train_mask=[6673], val_mask=[6673], test_mask=[6673])]}, '366_A1b_ASAP_tumor_map':{'train': [Data(x=[8481, 2048], edge_index=[2, 41449], y=[8481], pos=[8481, 2], train_mask=[8481], val_mask=[8481], test_mask=[8481])], 'val': [Data(x=[8178, 2048], edge_index=[2, 39998], y=[8178], pos=[8178, 2], train_mask=[8178], val_mask=[8178], test_mask=[8178])]}, '366_A1c_ASAP_tumor_map':{'train': [Data(x=[8477, 2048], edge_index=[2, 41553], y=[8477], pos=[8477, 2], train_mask=[8477], val_mask=[8477], test_mask=[8477])], 'val': [Data(x=[8539, 2048], edge_index=[2, 41785], y=[8539], pos=[8539, 2], train_mask=[8539], val_mask=[8539], test_mask=[8539])]}, '367_A2b_ASAP_tumor_map':{'train': [Data(x=[2290, 2048], edge_index=[2, 11108], y=[2290], pos=[2290, 2], train_mask=[2290], val_mask=[2290], test_mask=[2290]), Data(x=[2311, 2048], edge_index=[2, 11239], y=[2311], pos=[2311, 2], train_mask=[2311], val_mask=[2311], test_mask=[2311]), Data(x=[2053, 2048], edge_index=[2, 10031], y=[2053], pos=[2053, 2], train_mask=[2053], val_mask=[2053], test_mask=[2053]), Data(x=[2518, 2048], edge_index=[2, 12228], y=[2518], pos=[2518, 2], train_mask=[2518], val_mask=[2518], test_mask=[2518])], 'val': [Data(x=[2000, 2048], edge_index=[2, 9724], y=[2000], pos=[2000, 2], train_mask=[2000], val_mask=[2000], test_mask=[2000])]}, '368_A1b_ASAP_tumor_map':{'train': [Data(x=[14939, 2048], edge_index=[2, 73293], y=[14939], pos=[14939, 2], train_mask=[14939], val_mask=[14939], test_mask=[14939])], 'val': [Data(x=[14068, 2048], edge_index=[2, 68870], y=[14068], pos=[14068, 2], train_mask=[14068], val_mask=[14068], test_mask=[14068])]}, '368_A1c_ASAP_tumor_map':{'train': [Data(x=[16135, 2048], edge_index=[2, 79453], y=[16135], pos=[16135, 2], train_mask=[16135], val_mask=[16135], test_mask=[16135])], 'val': [Data(x=[15735, 2048], edge_index=[2, 76983], y=[15735], pos=[15735, 2], train_mask=[15735], val_mask=[15735], test_mask=[15735])]}, '368_A1d_ASAP_tumor_map':{'train': [Data(x=[17493, 2048], edge_index=[2, 86455], y=[17493], pos=[17493, 2], train_mask=[17493], val_mask=[17493], test_mask=[17493])], 'val': [Data(x=[16641, 2048], edge_index=[2, 81905], y=[16641], pos=[16641, 2], train_mask=[16641], val_mask=[16641], test_mask=[16641])]}, '369_A1b_ASAP_tumor_map':{'train': [Data(x=[4862, 2048], edge_index=[2, 23662], y=[4862], pos=[4862, 2], train_mask=[4862], val_mask=[4862], test_mask=[4862]), Data(x=[5344, 2048], edge_index=[2, 26102], y=[5344], pos=[5344, 2], train_mask=[5344], val_mask=[5344], test_mask=[5344])], 'val': [Data(x=[4344, 2048], edge_index=[2, 21160], y=[4344], pos=[4344, 2], train_mask=[4344], val_mask=[4344], test_mask=[4344])]}, '369_A1c_ASAP_tumor_map':{'train': [Data(x=[3579, 2048], edge_index=[2, 17421], y=[3579], pos=[3579, 2], train_mask=[3579], val_mask=[3579], test_mask=[3579]), Data(x=[2806, 2048], edge_index=[2, 13668], y=[2806], pos=[2806, 2], train_mask=[2806], val_mask=[2806], test_mask=[2806]), Data(x=[3465, 2048], edge_index=[2, 16751], y=[3465], pos=[3465, 2], train_mask=[3465], val_mask=[3465], test_mask=[3465])], 'val': [Data(x=[3144, 2048], edge_index=[2, 15240], y=[3144], pos=[3144, 2], train_mask=[3144], val_mask=[3144], test_mask=[3144])]}, '369_A2b_ASAP_tumor_map':{'train': [Data(x=[3510, 2048], edge_index=[2, 17166], y=[3510], pos=[3510, 2], train_mask=[3510], val_mask=[3510], test_mask=[3510]), Data(x=[3861, 2048], edge_index=[2, 18849], y=[3861], pos=[3861, 2], train_mask=[3861], val_mask=[3861], test_mask=[3861]), Data(x=[1230, 2048], edge_index=[2, 5880], y=[1230], pos=[1230, 2], train_mask=[1230], val_mask=[1230], test_mask=[1230]), Data(x=[1299, 2048], edge_index=[2, 6165], y=[1299], pos=[1299, 2], train_mask=[1299], val_mask=[1299], test_mask=[1299]), Data(x=[3534, 2048], edge_index=[2, 17090], y=[3534], pos=[3534, 2], train_mask=[3534], val_mask=[3534], test_mask=[3534])], 'val': [Data(x=[851, 2048], edge_index=[2, 4003], y=[851], pos=[851, 2], train_mask=[851], val_mask=[851], test_mask=[851])]}, '370_A1b_ASAP_tumor_map':{'train': [Data(x=[3522, 2048], edge_index=[2, 17230], y=[3522], pos=[3522, 2], train_mask=[3522], val_mask=[3522], test_mask=[3522]), Data(x=[3468, 2048], edge_index=[2, 16934], y=[3468], pos=[3468, 2], train_mask=[3468], val_mask=[3468], test_mask=[3468])], 'val': [Data(x=[3362, 2048], edge_index=[2, 16446], y=[3362], pos=[3362, 2], train_mask=[3362], val_mask=[3362], test_mask=[3362])]}, '370_A2a_ASAP_tumor_map':{'train': [Data(x=[3051, 2048], edge_index=[2, 14891], y=[3051], pos=[3051, 2], train_mask=[3051], val_mask=[3051], test_mask=[3051]), Data(x=[2460, 2048], edge_index=[2, 11918], y=[2460], pos=[2460, 2], train_mask=[2460], val_mask=[2460], test_mask=[2460])], 'val': [Data(x=[3383, 2048], edge_index=[2, 16557], y=[3383], pos=[3383, 2], train_mask=[3383], val_mask=[3383], test_mask=[3383])]}, '370_A2b_ASAP_tumor_map':{'train': [Data(x=[3236, 2048], edge_index=[2, 15876], y=[3236], pos=[3236, 2], train_mask=[3236], val_mask=[3236], test_mask=[3236]), Data(x=[3291, 2048], edge_index=[2, 16135], y=[3291], pos=[3291, 2], train_mask=[3291], val_mask=[3291], test_mask=[3291])], 'val': [Data(x=[3095, 2048], edge_index=[2, 15181], y=[3095], pos=[3095, 2], train_mask=[3095], val_mask=[3095], test_mask=[3095])]}, '37_A2d_ASAP_tumor_map':{'train': [Data(x=[7697, 2048], edge_index=[2, 36761], y=[7697], pos=[7697, 2], train_mask=[7697], val_mask=[7697], test_mask=[7697])], 'val': [Data(x=[7718, 2048], edge_index=[2, 36598], y=[7718], pos=[7718, 2], train_mask=[7718], val_mask=[7718], test_mask=[7718])]}, '61_A1a_ASAP_tumor_map':{'train': [Data(x=[1708, 2048], edge_index=[2, 8198], y=[1708], pos=[1708, 2], train_mask=[1708], val_mask=[1708], test_mask=[1708]), Data(x=[2059, 2048], edge_index=[2, 9967], y=[2059], pos=[2059, 2], train_mask=[2059], val_mask=[2059], test_mask=[2059]), Data(x=[2178, 2048], edge_index=[2, 10598], y=[2178], pos=[2178, 2], train_mask=[2178], val_mask=[2178], test_mask=[2178]), Data(x=[2096, 2048], edge_index=[2, 10164], y=[2096], pos=[2096, 2], train_mask=[2096], val_mask=[2096], test_mask=[2096])], 'val': [Data(x=[2201, 2048], edge_index=[2, 10721], y=[2201], pos=[2201, 2], train_mask=[2201], val_mask=[2201], test_mask=[2201])]}, '61_B1a_ASAP_tumor_map':{'train': [Data(x=[2145, 2048], edge_index=[2, 10425], y=[2145], pos=[2145, 2], train_mask=[2145], val_mask=[2145], test_mask=[2145]), Data(x=[212, 2048], edge_index=[2, 974], y=[212], pos=[212, 2], train_mask=[212], val_mask=[212], test_mask=[212]), Data(x=[2055, 2048], edge_index=[2, 9939], y=[2055], pos=[2055, 2], train_mask=[2055], val_mask=[2055], test_mask=[2055]), Data(x=[1275, 2048], edge_index=[2, 6075], y=[1275], pos=[1275, 2], train_mask=[1275], val_mask=[1275], test_mask=[1275]), Data(x=[1771, 2048], edge_index=[2, 8535], y=[1771], pos=[1771, 2], train_mask=[1771], val_mask=[1771], test_mask=[1771])], 'val': [Data(x=[2088, 2048], edge_index=[2, 10144], y=[2088], pos=[2088, 2], train_mask=[2088], val_mask=[2088], test_mask=[2088])]}, '70_A2b_ASAP_tumor_map':{'train': [Data(x=[1686, 2048], edge_index=[2, 8148], y=[1686], pos=[1686, 2], train_mask=[1686], val_mask=[1686], test_mask=[1686]), Data(x=[1663, 2048], edge_index=[2, 8025], y=[1663], pos=[1663, 2], train_mask=[1663], val_mask=[1663], test_mask=[1663]), Data(x=[1787, 2048], edge_index=[2, 8725], y=[1787], pos=[1787, 2], train_mask=[1787], val_mask=[1787], test_mask=[1787]), Data(x=[1805, 2048], edge_index=[2, 8797], y=[1805], pos=[1805, 2], train_mask=[1805], val_mask=[1805], test_mask=[1805]), Data(x=[1665, 2048], edge_index=[2, 8099], y=[1665], pos=[1665, 2], train_mask=[1665], val_mask=[1665], test_mask=[1665])], 'val': [Data(x=[1672, 2048], edge_index=[2, 8158], y=[1672], pos=[1672, 2], train_mask=[1672], val_mask=[1672], test_mask=[1672])]}, '7_A1c_ASAP_tumor_map':{'train': [Data(x=[6353, 2048], edge_index=[2, 31203], y=[6353], pos=[6353, 2], train_mask=[6353], val_mask=[6353], test_mask=[6353]), Data(x=[6222, 2048], edge_index=[2, 30556], y=[6222], pos=[6222, 2], train_mask=[6222], val_mask=[6222], test_mask=[6222])], 'val': [Data(x=[5921, 2048], edge_index=[2, 29101], y=[5921], pos=[5921, 2], train_mask=[5921], val_mask=[5921], test_mask=[5921])]}, '7_A1d_ASAP_tumor_map':{'train': [Data(x=[5494, 2048], edge_index=[2, 26986], y=[5494], pos=[5494, 2], train_mask=[5494], val_mask=[5494], test_mask=[5494]), Data(x=[5831, 2048], edge_index=[2, 28669], y=[5831], pos=[5831, 2], train_mask=[5831], val_mask=[5831], test_mask=[5831])], 'val': [Data(x=[5516, 2048], edge_index=[2, 27058], y=[5516], pos=[5516, 2], train_mask=[5516], val_mask=[5516], test_mask=[5516])]}, '7_A1e_ASAP_tumor_map':{'train': [Data(x=[4710, 2048], edge_index=[2, 23116], y=[4710], pos=[4710, 2], train_mask=[4710], val_mask=[4710], test_mask=[4710]), Data(x=[5083, 2048], edge_index=[2, 24985], y=[5083], pos=[5083, 2], train_mask=[5083], val_mask=[5083], test_mask=[5083])], 'val': [Data(x=[4639, 2048], edge_index=[2, 22765], y=[4639], pos=[4639, 2], train_mask=[4639], val_mask=[4639], test_mask=[4639])]}}

NameError: name 'Data' is not defined

In [14]:
import pickle
pickle.dump(graph_datasets,open('/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/graph_dataset.pkl','wb'))

In [17]:
e=pd.read_pickle("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/graph_dataset.pkl")

In [18]:
e

{'109_A1c_ASAP_tumor_map': {'train': [Data(x=[20589, 2048], edge_index=[2, 102219], y=[20589], pos=[20589, 2], train_mask=[20589], val_mask=[20589], test_mask=[20589])],
  'val': [Data(x=[20706, 2048], edge_index=[2, 102704], y=[20706], pos=[20706, 2], train_mask=[20706], val_mask=[20706], test_mask=[20706])]},
 '10_A1a_ASAP_tumor_map': {'train': [Data(x=[5563, 2048], edge_index=[2, 27309], y=[5563], pos=[5563, 2], train_mask=[5563], val_mask=[5563], test_mask=[5563]),
   Data(x=[5101, 2048], edge_index=[2, 24973], y=[5101], pos=[5101, 2], train_mask=[5101], val_mask=[5101], test_mask=[5101])],
  'val': [Data(x=[5804, 2048], edge_index=[2, 28532], y=[5804], pos=[5804, 2], train_mask=[5804], val_mask=[5804], test_mask=[5804])]},
 '10_A1b_ASAP_tumor_map': {'train': [Data(x=[5719, 2048], edge_index=[2, 28113], y=[5719], pos=[5719, 2], train_mask=[5719], val_mask=[5719], test_mask=[5719]),
   Data(x=[5649, 2048], edge_index=[2, 27761], y=[5649], pos=[5649, 2], train_mask=[5649], val_mask=[